In [ ]:
import os
import sys
os.add_dll_directory(r'C:\Users\modip\Downloads\openslide-bin-4.0.0.5-windows-x64\bin')
from openslide import open_slide
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata import WholeSlideImage
from wholeslidedata.interoperability.openslide.backend import OpenSlideWholeSlideImageBackend
import numpy as np
from matplotlib import pyplot as plt
import cv2
from patchify import patchify
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import uuid


In [ ]:
wsi = open_slide('ti\\t1\wsi\images\\100B.tif')
spacing = 0.45669399219053275 # more spacing means zoom out and less spacing means zoom in
real_spacing = wsi.get_real_spacing(spacing)
shape = wsi.get_shape_from_spacing(spacing)
downsampling = wsi.get_downsampling_from_spacing(spacing)

In [ ]:
print(f'wsi path: {wsi.path}')
print(f'level count: {wsi.level_count}')
print(f'spacings: {wsi.spacings}')
print(f'shapes:  {wsi.shapes}')
print(f'downsamplings: {wsi.downsamplings}')
print(f'closest real spacing to {spacing} = {real_spacing}')
print(f'shape at spacing {spacing} = {shape}')
print(f'downsampling at spacing {spacing} = {downsampling}')

In [ ]:
x, y = np.array(wsi.shapes[0]) / 2
width, height = 1024, 1024

slide8_0 = wsi.get_slide(spacing=3)
patch2_0 = wsi.get_patch(x=x, y=y, width=width, height=height, spacing=2.0)
patch0_5 = wsi.get_patch(x=x, y=y, width=width, height=height, spacing=0.5)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 10))
axes[0].imshow(patch0_5)
axes[0].set_title(f'patch at spacing 0.5 with center coordinate {x,y}', pad=20)
axes[1].imshow(patch2_0)
axes[1].set_title(f'patch at spacing 2.0 with center coordinate {x,y}', pad=20)
axes[2].imshow(slide8_0)
axes[2].set_title(f'slide at spacing 8.0', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
tc_wsi = WholeSlideImage(path='images\TC_S01_P000002_C0001_B107.tif', backend=OpenSlideWholeSlideImageBackend)
num_wsi=WholeSlideImage(path='images\\103S.tif', backend=OpenSlideWholeSlideImageBackend)
tc_slide = tc_wsi.get_slide(spacing=5.0)
num_slide = num_wsi.get_slide(spacing=8.0)
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
axes[0].imshow(tc_slide)
axes[0].set_title(f'TC Slide at spacing 1.0', pad=20)
axes[1].imshow(num_slide)
axes[1].set_title(f'NUM Slide at spacing 1.0', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
import cv2
import numpy as np

def reinhard_normalization(source, target):
    source_lab = cv2.cvtColor(source, cv2.COLOR_RGB2LAB).astype(np.float32)
    target_lab = cv2.cvtColor(target, cv2.COLOR_RGB2LAB).astype(np.float32)
    
    source_mean, source_std = cv2.meanStdDev(source_lab)
    target_mean, target_std = cv2.meanStdDev(target_lab)
    print(source_lab.shape)
    print(source_mean)
    print(source_std.shape)
    print(target_mean.shape)
    print(target_std.shape)
    source_lab = (source_lab - source_mean) * (target_std / source_std) + target_mean
    
    source_lab = np.clip(source_lab, 0, 255).astype(np.uint8)
    return cv2.cvtColor(source_lab, cv2.COLOR_LAB2RGB)

In [ ]:
tc_slide=tc_slide[:num_slide.shape[0],:num_slide.shape[1],:3]

In [ ]:
normalized_image = reinhard_normalization(tc_slide, num_slide)

In [ ]:
from tqdm import tqdm
from histomicstk.preprocessing.color_normalization.deconvolution_based_normalization import deconvolution_based_normalization
import cv2

In [ ]:
def do_convert(targets, sources, save_root):
    os.makedirs(save_root, exist_ok=True)
    for target in targets:
        target = cv2.imread(target)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
        for source in tqdm(sources):
            name = os.path.basename(source)

            source = cv2.imread(source)

            source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)

            stain_unmixing_routine_params = {
                'stains': ['hematoxylin', 'eosin'],
                'stain_unmixing_method': 'macenko_pca',
            }
            tissue_rgb_normalized = deconvolution_based_normalization(
                source,
                im_target=target,
                stain_unmixing_routine_params=stain_unmixing_routine_params
            )

            source_change = cv2.cvtColor(tissue_rgb_normalized, cv2.COLOR_RGB2BGR)
            result_path = os.path.join(save_root, name)
            cv2.imwrite(result_path, source_change)
    # os.makedirs(join(save_root), exist_ok=True)

In [ ]:
tgs = ['tgs.png',
        # '/media/totem_data_backup/totem/tiger-training-data/wsirois/roi-level-annotations/tissue-cells/images/114S_[25546, 18605, 26758, 19785].png'
    ]
scs = [
        'src.png',
        # '/media/totem_data_backup/totem/tiger-training-data/wsirois/roi-level-annotations/tissue-bcss/images/TCGA-LL-A73Y-01Z-00-DX1.50C20931-3AA9-40B4-8A73-56B1976423A8_[34061, 24725, 35279, 25785].png'
    ]
do_convert(targets=tgs, sources=scs, save_root='color_normalized')

In [ ]:


def process_image_pair(img_path, mask_path, img_dir, mask_dir, output_dir, patch_size):
    img = cv2.imread(os.path.join(img_dir, img_path))
    mask = cv2.imread(os.path.join(mask_dir, mask_path))
    
    if img is None or mask is None:
        return f"Failed to load {img_path} or {mask_path}"

    img_patches = patchify(img, (patch_size, patch_size, 3), step=patch_size).reshape(-1, patch_size, patch_size, 3)
    mask_patches = patchify(mask, (patch_size, patch_size, 3), step=patch_size).reshape(-1, patch_size, patch_size, 3)
    
    for i, (img_patch, mask_patch) in enumerate(zip(img_patches, mask_patches)):
        unique_id = uuid.uuid4().hex
        patch_filename = f'patch_{unique_id}.png'
        img_patch_path = os.path.join(output_dir, 'images', patch_filename)
        mask_patch_path = os.path.join(output_dir, 'masks', patch_filename)
        
        plt.imsave(img_patch_path, img_patch)
        plt.imsave(mask_patch_path, mask_patch)
    
    return f"Processed {img_path}"

def main():
    img_dir = 'roi_tissue\\bcss\\images'
    mask_dir = 'roi_tissue\\bcss\\masks'
    output_dir = 'patches'
    patch_size = 512

    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'masks'), exist_ok=True)

    img_paths = os.listdir(img_dir)
    mask_paths = os.listdir(mask_dir)

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(process_image_pair, img_path, mask_path, img_dir, mask_dir, output_dir, patch_size) 
                   for img_path, mask_path in zip(img_paths, mask_paths)]
        
        for future in tqdm(futures, total=len(futures), desc="Processing images"):
            print(future.result())

main()

In [ ]:
import os

# Specify the directory containing your files
directory = 'roi_tissue\\bcss\images'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.png') or filename.endswith('.jpg'):  # Adjust file extensions as needed
        # Remove square brackets and replace spaces with underscores
        new_filename = filename.replace('[', '').replace(']', '').replace(' ', '_')
        
        # Construct full file paths
        old_file = os.path.join(directory, filename)
        new_file = os.path.join('roi_tissue\\bcss\\reImages', new_filename)
        
        # Rename the file
        os.rename(old_file, new_file)
        print(f'Renamed: {filename} -> {new_filename}')

print("File renaming complete.")